# Dates and Times

Learning to manipulate and present dates and times within the SQL language can be crucial to fulfilling your reporting needs as many companies have a known standard as to the format in which they wish to present dates, to what accuracy they wish to see time data along with other requirements depending on who is looking at your reports and even where they are based (for example the differences in the way the US present dates compared to the UK).

#### What is the Date and Time right now?

Before we get into manipulating anything it's useful to know how to obtain the current date and time from within your SQL Server.

This is easy to do but it's worth noting that there are two variants you will likely see out in the wild and it's good to know why they exist.

We can see them both (and show that they return the same value) by executing the following:

In [ ]:
select current_timestamp, getDate();

So why are there two?  Well, again this comes back to the vendor making the language their own.

<code>GetDate()</code> is a function which is unique to the T-SQL language and therefore Microsoft's own implementation.  It actually, in the background, calls current_timestamp which is why they return exactly the same data.

<code>Current_timestamp</code> is from the generic SQL language and therefore will work in all SQL based products.

My recommendation is to use the latter because this will make your code more widely understandable and portable if ever needed.  Using the purely T-SQL version means that your code will only work on Microsoft SQL Server and ensures it is harder to migrate your code if ever needed.

#### DATEPART

We shall now move on to a couple of the questions that were posed in the introduction to Data Manipulation:
* **Year of order**
* **Day of Week order was taken**
* Employee Name
* Customer's Full Name
* Total Order Value (Sale Amount + Tax)

There is a provided function in SQL which allows us to specify which part of the datetime we wish to focus on, and that function is <code>DATEPART</code>.

It is used as follows:
<code>DATEPART(portionOfDateTime, dateTimeValue)</code>

Some of the key 'portions' we can use (these are the most applicable for DATEPART, but we shall come across a few more shortly) are:
* 'yy' / 'yyyy' / 'year'
 * Any of these result in the year of the date passed in
* 'MM' / 'month'
 * This will give you the month portion of the date
* 'dd' / 'day'
 * The day portion of the date that has been passed in
* 'dw'
 * This is the day of the week
 * It will be in a format of 1 - 7
 * Please test on your system to know which number corresponds to which day
  * These can vary depending on the language settings of your SQL Server

Using our test table we can see these in action, specifically for OrderID = 1:

In [ ]:
select orderID, orderDate,
		datepart(yy, orderDate),
		datepart(MM, orderDate),
		datepart(dd, orderDate),
		datepart(dw, orderDate)
from JupyterDatabase.sales.basicSales
where orderID = 1;

You can see that we have some of the results that we require for our prospective query.

We can also utilise this function within the WHERE clause of our query as required.  OrderID 1 was placed on day of the week 6 (which translates to Friday on my SQL Server install), so let's have a look to see if any other orders were placed on a Friday:

In [ ]:
select *
from JupyterDatabase.sales.basicSales
where datepart(dw, orderDate) = 6;

As you can see, there are 3 orders placed on a Friday.

There are several other date manipulation functions that should also be mentioned at this point.

#### DATEADD

The purpose of this function is to easily add and subtract date periods from a date.  This is therefore incredibly useful within the WHERE clause to satisfy requirements such as "how many orders in the last week".  Using this function we can create a query that is re-usable every day without ever changing the code.  I shall show this shortly.

The <code>DATEADD</code> function is defined as follows:
<code>DATEADD(portionOfDateTime, numberOfPortions, dateTimeValue)</code>

The additional date portions we are going to introduce here are:
* 'ww' / 'week'
 * This displays the week of the year from 1 to 52
* 'hh' / 'hour'
 * The hour portion of the time
* 'n', 'minute'
 * This is the minute.  Due to MM being month, we utilise 'n' as minute
* 's' / 'second'
 * Displays the seconds of the datetime value
* 'ms' / 'millisecond'
 * This manipulates the millisecond portion of the datetime value

To show a couple of these in action we can add a year and minute respectively to the orderDate of OrderID 1:

In [ ]:
select orderID, orderDate,
		dateadd(yy, 1, orderDate),
		dateadd(n, 1, orderDate)
from JupyterDatabase.sales.basicSales
where orderID = 1;

As you can see, this is how we manipulate the addition and subtraction of time periods to a datetime.

The most common usage for this would be that which I mentioned above, where we can create purely re-usable code by asking SQL Server to perform WHERE orderDate between now and 7 days ago.

In [ ]:
select *
from JupyterDatabase.sales.basicSales
where orderDate between dateadd(dd, -7, current_timestamp) and current_timestamp;

Note this will not return any data as the table is not set up to support a rolling 7 days of information (this is a static document after all), but the coding principle remains solid.

It is also worth noting that, as you can see, it is possible and valid to place a negative number into the dateadd function.

#### DATEDIFF

As the name implies with this function, we are now going to determine the difference between two dates.

<code>DATEDIFF(portionOfDatetime, startDateTime, endDateTime)</code>

The portions we will use are exactly the same as all those we have mentioned so far as we have now covered them all above, therefore we can move swiftly into a demo.

In the following statement we will look for the difference, in days, between two dates and also the difference in minutes between those same dates:

In [ ]:
select datediff(dd, '2020-01-01', '2020-01-03'),
		datediff(s, '2020-01-01', '2020-01-03');

#### Generic Conversions

The functions mentioned above are useful when specifying portions of the date and time that we are after and using a stable syntax.

However, there are also more basic methods of obtaining parts of a date.  Feel free to use whichever you feel more comfortable with at the time:

In [ ]:
select '2020-07-31 12:37:59.123',
		year('2020-07-31 12:37:59.123'),
		month('2020-07-31 12:37:59.123'),
		day('2020-07-31 12:37:59.123');

Note that the above is a definitive list.  There are no functions of this type to cater for HOUR, MINUTE, SECOND etc.  Hence for consistency I tend to use the DATEPART function.